# Data Prepariation

In [1]:
import tensorflow as tf
import utils
import numpy as np
import tensorflow as tf
import argparse
import os
import time
import pickle

from model import Model
from utils import DataLoader
from tensorflow.python.ops import variable_scope


In [2]:
data_loader=utils.DataLoader()
sess=tf.InteractiveSession()

In [3]:
sess.run(tf.global_variables_initializer())

# Functions

In [4]:
def rnn_decoder(decoder_inputs,
                initial_state,
                cell,
                loop_function=None,
                scope=None):
    with variable_scope.variable_scope(scope or "rnn_decoder"):
        state = initial_state
        outputs = []
        prev = None
        for i, inp in enumerate(decoder_inputs):
            if loop_function is not None and prev is not None:
                with variable_scope.variable_scope("loop_function", reuse=True):
                    inp = loop_function(prev, i)
            if i > 0:
                variable_scope.get_variable_scope().reuse_variables()
            output, state = cell(inp, state)
            outputs.append(output)
            if loop_function is not None:
                prev = output
    return outputs, state

# Model

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.python.ops import rnn_cell
class Model():

    def __init__(self):
        '''
        parameters_init
        '''
        sequence_length=data_loader.seq_length
        rnn_size=130
        num_layers=3
        self.lr=0.00001
        batch_size=50
        embedding_size=32
        grad_clip=5
        '''
        data_input
        '''
        self.input_data = tf.placeholder(tf.float32, [None,sequence_length, 2])
        #input:(batch_size,序列长度，2)
        self.target_data = tf.placeholder(tf.float32, [None, sequence_length, 2])
        #target_data:(batch_size,序列长度，2)
        seq_length=int(5)
        self.inputs = tf.split(axis=1, num_or_size_splits=int(seq_length), value=self.input_data)
        #(序列长度，batch_size,1,2)
        self.inputs = [tf.squeeze(input_, [1]) for input_ in self.inputs]
        #(序列长度，batch_size,2)
        #即转成了每一列为一个batch的一组序列
        # Get a list of 2D tensors. Each of size numPoints x 2
        '''
        embedding
        '''
        self.embedded_inputs = []
        #with tf.variable_scope("coordinate_embedding"):
        with tf.variable_scope("coordinate_embedding",reuse=True):
            #  The spatial embedding using a ReLU layer
            #  Embed the 2D coordinates into embedding_size dimensions
            #  TODO: (improve) For now assume embedding_size = rnn_size
            embedding_w = tf.get_variable("embedding_w", [2, embedding_size])
            embedding_b = tf.get_variable("embedding_b", [embedding_size])       
        for x in self.inputs:
            #这里的x跟坐标的x没关系，只是指代一个
            # Each x is a 2D tensor of size numPoints x 2
            # Embedding layer
            embedded_x = tf.nn.relu(tf.add(tf.matmul(x, embedding_w), embedding_b))
            self.embedded_inputs.append(embedded_x)
        #得到的embedded_inputs维度是（sequence_length，batch_size,embedding_size）
            
        
        '''
        rnn_cell defination
        '''
        cell=rnn_cell.BasicLSTMCell(rnn_size,state_is_tuple=True)
        #cell=rnn_cell.MultiRNNCell([cell]*num_layers,state_is_tuple=True)
        self.cell=cell
        self.initial_state=cell.zero_state(batch_size=batch_size,dtype=tf.float32)       
        # Output size is the set of parameters (mu, sigma, corr)
        output_size = 5  # 2 mu, 2 sigma and 1 corr
        #with tf.variable_scope("rnnlm"):
        with tf.variable_scope("rnnlm",reuse=True):
            output_w = tf.get_variable("output_w", [rnn_size, output_size], initializer=tf.truncated_normal_initializer(stddev=0.01), trainable=True)
            output_b = tf.get_variable("output_b", [output_size], initializer=tf.constant_initializer(0.01), trainable=True)
                 
        #with tf.variable_scope('rnnlm'):
        with tf.variable_scope('rnnlm',reuse=True):
            self.outputs, last_state =rnn_decoder(self.embedded_inputs, self.initial_state, cell, loop_function=None)
            #outputs shape[sequence_length, batch_size, hidden_layers]
        self.output = tf.reshape(tf.concat(self.outputs,1), [-1,rnn_size])
        #output shape: [sequence*batch_size,hidden_layers]
        self.output = tf.nn.xw_plus_b(self.output, output_w, output_b)
        
        self.final_state = last_state
        #final_state shape: [2(两个隐层数组)，batch_size,hidden_layers)]
        flat_target_data = tf.reshape(self.target_data, [-1, 2])
        [x_data, y_data] = tf.split(flat_target_data, 2,1)
        #x_data代表target_data的第一列，即所有x坐标
        #y_data代表target_data的第二列，即所有y坐标
        '''
        some built_in_class functions 
        '''
        def tf_2d_normal(x, y, mux, muy, sx, sy, rho):
            #输入x,y输出其对应的概率密度值是多少
            '''x : input x points
            y : input y points
            mux : mean of the distribution in x
            muy : mean of the distribution in y
            sx : std dev of the distribution in x
            sy : std dev of the distribution in y
            rho : Correlation factor of the distribution'''
            normx = tf.subtract(x, mux)
            normy = tf.subtract(y, muy)
            sxsy = tf.multiply(sx, sy)
            z = tf.square(tf.div(normx, sx)) + tf.square(tf.div(normy, sy)) - 2*tf.div(tf.multiply(rho, tf.multiply(normx, normy)), sxsy)
            negRho = 1 - tf.square(rho)
            result = tf.exp(tf.div(-z, 2*negRho))
            denom = 2 * np.pi * tf.multiply(sxsy, tf.sqrt(negRho))
            result = tf.div(result, denom)
            self.result = result
            return result
        
        def get_lossfunc(z_mux, z_muy, z_sx, z_sy, z_corr, x_data, y_data):
            step = tf.constant(1e-3, dtype=tf.float32, shape=(1, 1))
            # Calculate the PDF of the data w.r.t to the distribution
            result0_1 = tf_2d_normal(x_data, y_data, z_mux, z_muy, z_sx, z_sy, z_corr)
            result0_2 = tf_2d_normal(tf.add(x_data, step), y_data, z_mux, z_muy, z_sx, z_sy, z_corr)
            result0_3 = tf_2d_normal(x_data, tf.add(y_data, step), z_mux, z_muy, z_sx, z_sy, z_corr)
            result0_4 = tf_2d_normal(tf.add(x_data, step), tf.add(y_data, step), z_mux, z_muy, z_sx, z_sy, z_corr)
            result0 = tf.div(tf.add(tf.add(tf.add(result0_1, result0_2), result0_3), result0_4), tf.constant(4.0, dtype=tf.float32, shape=(1, 1)))
            #数值稳定性，加小系数求平均
            result0 = tf.multiply(tf.multiply(result0, step), step)
            # For numerical stability purposes
            epsilon = 1e-20
            result1 = -tf.log(tf.maximum(result0, epsilon))  # Numerical stability
            return tf.reduce_sum(result1)
    
        def get_coef(output):
            z = output
            z_mux, z_muy, z_sx, z_sy, z_corr = tf.split(z,5,1)
            z_sx = tf.exp(z_sx)
            z_sy = tf.exp(z_sy)
            z_corr = tf.tanh(z_corr)
            return [z_mux, z_muy, z_sx, z_sy, z_corr]   
        [o_mux, o_muy, o_sx, o_sy, o_corr] = get_coef(self.output)
        # Store the predicted outputs
        self.mux = o_mux
        self.muy = o_muy
        self.sx = o_sx
        self.sy = o_sy
        self.corr = o_corr
        # Compute the loss function
        lossfunc = get_lossfunc(o_mux, o_muy, o_sx, o_sy, o_corr, x_data, y_data)
        self.cost = tf.div(lossfunc, (batch_size * seq_length))
        tvars = tf.trainable_variables()
        self.gradients = tf.gradients(self.cost, tvars)
        grads, _ = tf.clip_by_global_norm(self.gradients,grad_clip)
        optimizer = tf.train.RMSPropOptimizer(self.lr)
        with tf.variable_scope("rnnlm",reuse=True):
        #with tf.variable_scope("rnnlm"):
            self.train_op = optimizer.apply_gradients(zip(grads, tvars))
        
    def sample(self, sess, traj, num=10):
        '''
        Given an initial trajectory (as a list of tuples of points), predict the future trajectory
        until a few timesteps
        Params:
        sess: Current session of Tensorflow
        traj: List of past trajectory points
        num: Number of time-steps into the future to be predicted
        '''
        def sample_gaussian_2d(mux, muy, sx, sy, rho):
            '''
            Function to sample a point from a given 2D normal distribution
            params:
            mux : mean of the distribution in x
            muy : mean of the distribution in y
            sx : std dev of the distribution in x
            sy : std dev of the distribution in y
            rho : Correlation factor of the distribution
            '''
            # Extract mean
            mean = [mux, muy]
            # Extract covariance matrix
            cov = [[sx*sx, rho*sx*sy], [rho*sx*sy, sy*sy]]
            # Sample a point from the multivariate normal distribution
            x = np.random.multivariate_normal(mean, cov, 1)
            return x[0][0], x[0][1]

        # Initial state with zeros
        state = sess.run(self.cell.zero_state(1, tf.float32))

        # Iterate over all the positions seen in the trajectory
        for pos in traj[:-1]:
            # Create the input data tensor
            data = np.zeros((1, 1, 2), dtype=np.float32)
            data[0, 0, 0] = pos[0]  # x
            data[0, 0, 1] = pos[1]  # y

            # Create the feed dict
            feed = {self.input_data: data, self.initial_state: state}
            # Get the final state after processing the current position
            [state] = sess.run([self.final_state], feed)

        ret = traj

        # Last position in the observed trajectory
        last_pos = traj[-1]

        # Construct the input data tensor for the last point
        prev_data = np.zeros((1, 1, 2), dtype=np.float32)
        prev_data[0, 0, 0] = last_pos[0]  # x
        prev_data[0, 0, 1] = last_pos[1]  # y

        for t in range(num):
            # Create the feed dict
            feed = {self.input_data: prev_data, self.initial_state: state}

            # Get the final state and also the coef of the distribution of the next point
            [o_mux, o_muy, o_sx, o_sy, o_corr, state] = sess.run([self.mux, self.muy, self.sx, self.sy, self.corr, self.final_state], feed)

            # Sample the next point from the distribution
            next_x, next_y = sample_gaussian_2d(o_mux[0][0], o_muy[0][0], o_sx[0][0], o_sy[0][0], o_corr[0][0])
            # Append the new point to the trajectory
            ret = np.vstack((ret, [next_x, next_y]))

            # Set the current sampled position as the last observed position
            prev_data[0, 0, 0] = next_x
            prev_data[0, 0, 1] = next_y

        return ret



In [23]:
model=1
model=Model()
saver = tf.train.Saver(tf.all_variables())
sess.run(tf.global_variables_initializer())

Instructions for updating:
Please use tf.global_variables instead.


In [24]:
saver.restore(sess, 'save/model.ckpt-2500')

INFO:tensorflow:Restoring parameters from save/model.ckpt-2500


# Train

In [25]:
#model=1
#model=Model()
#sess.run(tf.global_variables_initializer())
epochs=100
for e in range(epochs):
            data_loader.reset_batch_pointer()
            # Get the initial cell state of the LSTM
            #state=sess.run(model.initial_state)
            #saver = tf.train.Saver(tf.all_variables())
            # For each batch in this epoch
            for b in range(data_loader.num_batches):
                start = time.time()
                x, y = data_loader.next_batch()
                # Feed the source, target data and the initial LSTM state to the model
                feed = {model.input_data: x, model.target_data: y,model.initial_state: state}
                # Fetch the loss of the model on this batch, the final LSTM state from the session
                train_loss, state, _ = sess.run([model.cost, model.final_state, model.train_op], feed)
                end = time.time()
                print("{}/{} (epoch {}), train_loss = {:.3f}, time/batch = {:.3f}"
                    .format(
                        e * data_loader.num_batches + b,
                        50 * data_loader.num_batches,
                        e,
                        train_loss, end - start))
            
                if (e * data_loader.num_batches + b) % 500 == 0 and ((e * data_loader.num_batches + b) > 0):
                    checkpoint_path = os.path.join('save', 'model.ckpt')
                    saver.save(sess, checkpoint_path, global_step=e * data_loader.num_batches + b)
                    print("model saved to {}".format(checkpoint_path))

0/2800 (epoch 0), train_loss = 10.110, time/batch = 0.871
1/2800 (epoch 0), train_loss = 9.879, time/batch = 0.031
2/2800 (epoch 0), train_loss = 9.411, time/batch = 0.031
3/2800 (epoch 0), train_loss = 10.995, time/batch = 0.038
4/2800 (epoch 0), train_loss = 9.207, time/batch = 0.031
5/2800 (epoch 0), train_loss = 11.112, time/batch = 0.047
6/2800 (epoch 0), train_loss = 9.220, time/batch = 0.046
7/2800 (epoch 0), train_loss = 10.943, time/batch = 0.047
8/2800 (epoch 0), train_loss = 8.884, time/batch = 0.047
9/2800 (epoch 0), train_loss = 11.829, time/batch = 0.037
10/2800 (epoch 0), train_loss = 9.316, time/batch = 0.034
11/2800 (epoch 0), train_loss = 10.967, time/batch = 0.035
12/2800 (epoch 0), train_loss = 9.054, time/batch = 0.042
13/2800 (epoch 0), train_loss = 10.649, time/batch = 0.053
14/2800 (epoch 0), train_loss = 8.904, time/batch = 0.052
15/2800 (epoch 0), train_loss = 11.226, time/batch = 0.038
16/2800 (epoch 0), train_loss = 8.963, time/batch = 0.036
17/2800 (epoch 0

In [ ]:
np.array(mux).shape

In [15]:
??tf.contrib